Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

Load and visualize dataset

In [ ]:
#Load data
train_data = pd.read_csv('Train.csv')

#Split data and labels
X = train_data.iloc[:, 1:]  
y = train_data.iloc[:, 0]

#Process data
if not isinstance(X, pd.DataFrame):
    X = pd.DataFrame(X)
X = X.apply(pd.to_numeric, errors='coerce')
X = X.fillna(0)  
X = X.values / 255.0
X = X.reshape(-1, 28, 28, 1)

#Visualize data as image on chosen index
plt.imshow(X[1].reshape(28, 28), cmap='gray')

ACTIVATION FUNCTION 


USE ReLU Functions to introduce non-linearity for model can do multi-class classification

In [ ]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = np.random.rand(n_inputs, n_neurons)
        self.bias    = np.zeros((1,n_neurons))

class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0,inputs)

SOFTMAX

In [ ]:
class Activation_Softmax:
    def forward(self, inputs):
            exp_values = np.exp(inputs - np.max(inputs))
            prob       = exp_values/np.sum(exp_values, axis=1,keepdims=True)
            self.output = prob

LOSS AND ENTROPY 

BACKPROP 

OPTIMIZER

LEARNING AND TRAINING 

CONVOLUTION

PARAMETERS:
+ Image  : 3D ARRAY
+ K      : Kernel (or filters) small matrix slide on images pixel 
+ padding: The ammount pixel 
+ stride : jump of the kernel 

In [ ]:
def my_Convolution(image_array, K, padding=0, stride=1):
    if image_array.ndim == 2:
        is_grayscale = True
        # Thêm chiều kênh màu: (cao, rộng) -> (cao, rộng, 1)
        image_array = np.expand_dims(image_array, axis=2)
    else:
        is_grayscale = False

    # Lấy kích thước của hình ảnh ban đầu (bây giờ luôn là 3D)
    [xImgShape, yImgShape, numChan] = image_array.shape
    # Lấy kích thước của kernel
    [xK, yK] = K.shape

    # Sử dụng công thức tích chập để tính kích thước đầu ra
    xOutput = int((xImgShape - xK + 2 * padding) / stride + 1)
    yOutput = int((yImgShape - yK + 2 * padding) / stride + 1)
    
    # Tạo mảng rỗng để chứa kết quả
    output = np.zeros((xOutput, yOutput, numChan))
    
    # Tạo ảnh có đệm (padding)
    imagePadded = np.zeros((xImgShape + 2 * padding, yImgShape + 2 * padding, numChan))
    imagePadded[padding:padding + xImgShape, padding:padding + yImgShape, :] = image_array
    
    # Vòng lặp thực hiện tích chập
    for c in range(numChan):  
        for y in range(yOutput):
            for x in range(xOutput):
                # Xác định tọa độ của vùng cắt
                y_start = y * stride
                y_end = y_start + yK
                x_start = x * stride
                x_end = x_start + xK

                # Cắt slice theo đúng thứ tự (hàng, cột, kênh)
                current_slice = imagePadded[y_start:y_end, x_start:x_end, c]
                
                # Nhân theo từng phần tử và tính tổng
                s = np.multiply(current_slice, K)
                output[x, y, c] = np.sum(s)
    
    # Hiển thị kết quả
    # Nếu là ảnh màu, ta cộng các kênh lại để hiển thị ảnh xám.
    # Nếu là ảnh xám, ta chỉ cần hiển thị kênh duy nhất đó.
    if is_grayscale:
        display_image = output.squeeze() # Bỏ chiều kênh: (cao, rộng, 1) -> (cao, rộng)
    else:
        display_image = output.sum(axis=2) # Cộng các kênh màu
        
    plt.imshow(display_image, cmap='gray')
    plt.title(f'Sau tich chap (padding={padding}, stride={stride})')
    plt.show()
    
    # Trả về kết quả, nếu là ảnh xám thì trả về mảng 2D cho nhất quán
    if is_grayscale:
        return output.squeeze()
    else:
        return output

POOLING

In [ ]:
def Max_Pooling(input_array, kernel_size=2, stride=1):
    k, s = kernel_size, stride
    
    # Calculate output shape
    output_shape = ((input_array.shape[0] - kernel_size) // stride) + 1
    
    # Initialize output array with correct shape
    output = np.zeros((output_shape, output_shape))
    
    # Perform max pooling
    for i in range(output_shape):
        for j in range(output_shape):
            # Extract the window
            window = input_array[i*s:i*s + k, j*s:j*s + k]
            # Get max value from window
            output[i][j] = np.max(window)
    
    return output



SIGMOID AND TANH

FLATTENING

BACKPROB AGAIN 

TESTING

MODEL LAYOUT
- Input : images (resolution of 28 x 28 and monochromatic) of hand written digits
- Block 1
    + Conv2D (1)
    + Activation (1)
    + Max Pooling (1)
- Block 2
    + Conv2D (2)
    + Activation (2)
    + Max Pooling (2)
- Flatten
- Block 3
    + Fully Connected (1)
    + Activation (3)
- Fully Connected
- Softmax